In [15]:
#!/usr/bin/env python
import json, re, sys, os, json, subprocess, time, logging, requests, urllib3
from subprocess import call, check_output
from requests.structures import CaseInsensitiveDict
urllib3.disable_warnings()
import requests

#The AppD secret is passed in via Env Var which is injected into the build container via the pipeline
#import the env vars and logon to vault to get the secret and then use it to run this command and write the output to the vault
#secret = os.getenv('APPD_SECRET')

APPD_SECRET = '4f29e0b6-81ba-46cd-a4d0-8aecc825f712'

secret = '22ae7cb8-61cc-4ac6-afd6-4a39a12b9259'

'''
#curl -X POST -H “Content-Type: application/vnd.appd.cntrl+protobuf;v=1” “https://<controller address>/controller/api/oauth/access_token” -d ‘grant_type=client_credentials&client_id=<userName>@<accountName>&client_secret=face10d5-573e-4a75-8396-afa006fd8f19’
url = "https://cisco-apipartnertraininglab.saas.appdynamics.com/auth/v1/oauth/token"
payload = 'grant_type=client_credentials&client_id=fsolab4%40cisco-apipartnertraininglab_secret=' + secret
print(payload)
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.json)

'''

import requests

url = "https://cisco-apipartnertraininglab.saas.appdynamics.com/auth/v1/oauth/token"

payload='grant_type=client_credentials&client_id=sconrod%40cisco-apipartnertraininglab&client_secret=22ae7cb8-61cc-4ac6-afd6-4a39a12b9259'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)











'''

token_json = response.json()
print(token_json)


TOKEN = token_json['access_token']
print("This it the temp oath token generated for appd")

#write the token to a json file called data.json
outfile = 'token.json'
with open(outfile, 'w') as my_file:
    my_file.write(TOKEN)
my_file.close()


#write the appd oath token returned from the api call to the vault

url = "http://prod-vault.devops-ontap.com:8200/v1/concourse/cisco-fso-labs/appd-oath"

VAULT_ADDR = os.getenv('VAULT_ADDRR')
VAULT_TOKEN = os.getenv('VAULT_TOKEN')

headers = CaseInsensitiveDict()
headers = {"X-Vault-Token":VAULT_TOKEN}
headers = {"Content-Type: application/json"}
data = '{"data":{"token":TOKEN}'
resp = requests.post(url=url,data=data)
print(resp.text)
print(type(resp.text))


#Here we are writing the temporary token for AppDynamics into the vault, so we may call it as a variable in subsequent API calls.
#we need to put this in a module form, so that each time we perform API calls we get a new token
import requests
from requests.structures import CaseInsensitiveDict

url = "http://prod-vault.devops-ontap.com:8200/v1/concourse/cisco-fso-labs/appd-oath"

headers = CaseInsensitiveDict()
headers["X-Vault-Token"] = VAULT_TOKEN
headers["Content-Type"] = "application/json"

data_json = {"token": TOKEN}

resp = requests.post(url, headers=headers, json=data_json)

print(resp.status_code)
'''

{"error":"invalid_client","error_description":"Unable to validate client credentials."}


'\n\ntoken_json = response.json()\nprint(token_json)\n\n\nTOKEN = token_json[\'access_token\']\nprint("This it the temp oath token generated for appd")\n\n#write the token to a json file called data.json\noutfile = \'token.json\'\nwith open(outfile, \'w\') as my_file:\n    my_file.write(TOKEN)\nmy_file.close()\n\n\n#write the appd oath token returned from the api call to the vault\n\nurl = "http://prod-vault.devops-ontap.com:8200/v1/concourse/cisco-fso-labs/appd-oath"\n\nVAULT_ADDR = os.getenv(\'VAULT_ADDRR\')\nVAULT_TOKEN = os.getenv(\'VAULT_TOKEN\')\n\nheaders = CaseInsensitiveDict()\nheaders = {"X-Vault-Token":VAULT_TOKEN}\nheaders = {"Content-Type: application/json"}\ndata = \'{"data":{"token":TOKEN}\'\nresp = requests.post(url=url,data=data)\nprint(resp.text)\nprint(type(resp.text))\n\n\n#Here we are writing the temporary token for AppDynamics into the vault, so we may call it as a variable in subsequent API calls.\n#we need to put this in a module form, so that each time we perfo